# Obteniendo el **TOTAL de vacunados y fallecidos** por COVID-19 por cada **semana epidemiológica** de todo el Perú

## 0. Cargar librerías

In [21]:
import pandas as pd
import numpy as np

import functions as fn

## 1. Cargamos direcciones de RawData

In [24]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesamos el dataset de fallecidos

### 2.1. Leemos el dataset

In [25]:
fal_col = ['FECHA_FALLECIMIENTO']
df_fal = fn.read_largeCSV_file(fal_url, ';', fal_col)
print(df_fal.head())

   FECHA_FALLECIMIENTO
0             20210425
1             20210428
2             20210430
3             20210421
4             20210803


### 2.2. Procesamos el dataset

In [26]:
fn.variable_fecha(df_fal, 'FECHA_FALLECIMIENTO') # Formato fecha en columna 'FECHA_FALLECIMIENTO'
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO') # Obtenemos año y semana epidemiológica de fallecidos  

df_fal['fallecido'] = 1 # Contador de casos (cada 1 es un fallecido)                                      
df_fal['fallecido'].apply(np.int8)

# Devuelve un dataframe con el total de FALLECIDOS por semana y año epidemiológico
epi_fal = pd.crosstab(index=[df_fal['epi_year'], df_fal['epi_week']],
                      columns=df_fal['fallecido'])

epi_fal.columns = ['fallecidos'] # Renombramos

del df_fal
print(epi_fal.head())

                   fallecidos
epi_year epi_week            
2020     10                 2
         11                 3
         12                30
         13                62
         14               223


## 3. Procesamos el dataset de vacunados

### 3.1. Leemos el dataset

In [27]:
vac_col = ['fecha_vacunacion','dosis']  # Seleccionar solamente la columna de fechas de ambos datasets 
df_vac = fn.read_largeCSV_file(vac_url, ',', vac_col)   # Leemos los datasets 
print(df_vac.head())

  fecha_vacunacion  dosis
0       19/07/2021      2
1       17/06/2021      2
2       11/06/2021      2
3       28/07/2021      2
4        8/05/2021      1


### 3.2. Procesamos el dataset

In [28]:
# Convertimos el dataset de vacunados en una lista de chunks o dfs
# (Para procesar información en bloques de 500 000) 
lst_vac = fn.df_into_chunks(df_vac)
del df_vac

# Que cada 'chunk' de lst_vac sea solamente cuando dosis = 2, es decir tengan 2 dosis
lst_vac = [chunk.loc[chunk.loc[:, 'dosis'] == 2] for chunk in lst_vac]

# Convertir a formato fecha la columna 'fecha vacunacion' y obtener su año y semana epi
[fn.variable_fecha_ymd(chunk, 'fecha_vacunacion') for chunk in lst_vac]
[fn.date_to_epiweek(chunk, 'fecha_vacunacion') for chunk in lst_vac]

# Creamos columnas de 1 en cada chunk para contabilizar cada caso de vacunado
for chunk in lst_vac:                                        
    chunk['vacunado'] = 1
    chunk['vacunado'].apply(np.int8)
    del chunk['dosis']  # Borramos columna dosis llena de número 2
del chunk # Borramos el último chunk que queda al final

print(lst_vac[0].head())

   epi_year  epi_week  vacunado
0      2021        29         1
1      2021        24         1
2      2021        23         1
3      2021        30         1
7      2021        22         1


In [29]:
def epiweeks_chunks(dfs_vac):
    """
    Devuelve un dataframe con el total de VACUNADOS por semana y año 
    epidemiológico (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index=[chunk['epi_year'],
                                                            chunk['epi_week']],
                                                     columns=chunk['vacunado'])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])
        
    # Unimos todos los dfs sumados en uno solo
    merged_epivac = pd.concat(lst_epi_vac, axis=1) 
    epi_vac = pd.DataFrame(merged_epivac.sum(numeric_only=True, axis=1))
    epi_vac.columns = ['vacunados']
    
    return epi_vac

epi_vac = epiweeks_chunks(lst_vac)
del lst_vac
print(epi_vac.head())


                   vacunados
epi_year epi_week           
2021     6               1.0
         7               4.0
         8              19.0
         9           93044.0
         10          80891.0


## 4. Unimos el dataset del TOTAL de vacunados (2 dosis) y TOTAL de fallecidos por su semana epidemiológica y año

In [30]:
# Concatenamos ambos dataframes por columnas
merged_epiweeks = pd.concat([epi_fal, epi_vac], axis=1)         
merged_epiweeks = merged_epiweeks.fillna(value = 0) # Rellenamos vacíos o Nan values con 0

# Cambiamos a Int64 ya que existen Nan values y cambia a float64 automáticamente
merged_epiweeks = merged_epiweeks.astype('Int64')
merged_epiweeks = merged_epiweeks.rename({'fallecidos': 'deceased', 
                                          'vacunados': 'vaccinated'}, axis=1) # Renombrar columnas
                                          
print(merged_epiweeks.head())

                   deceased  vaccinated
epi_year epi_week                      
2020     10               2           0
         11               3           0
         12              30           0
         13              62           0
         14             223           0


## 5. Guardar el dataframe

In [20]:
merged_epiweeks.to_csv('Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv')